In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4161,4162,15611371,Siciliani,736,France,Male,43,4,176134.54,1,1,1,52856.88,0
8533,8534,15768566,K?,706,France,Male,34,8,0.00,2,1,1,37479.97,0
6965,6966,15656188,Davis,584,Spain,Female,30,5,0.00,2,1,1,185201.58,0
7075,7076,15768295,Warner,778,France,Female,34,7,109564.10,1,0,1,113046.81,0
8841,8842,15718769,Fallaci,557,Spain,Male,36,1,113110.26,1,1,0,98413.10,0


In [4]:
data.drop(columns=["RowNumber", "CustomerId", "Surname"],axis=1, inplace = True)

In [5]:
data.sample()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2578,582,France,Female,32,1,116409.55,1,0,1,152790.92,0


In [6]:
## encode one categorical variable
labe_encoder_gender = LabelEncoder()

data['Gender'] = labe_encoder_gender.fit_transform(data['Gender'])

In [7]:
data.sample()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8075,850,France,1,58,8,156652.13,1,0,0,25899.21,1


In [9]:
## OHE geographic cols
from sklearn.preprocessing import OneHotEncoder

ohe_geo = OneHotEncoder()

geo_encoder = ohe_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=ohe_geo.get_feature_names_out(['Geography'])).astype(int)

In [13]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1,0,0
1,0,0,1
2,1,0,0
3,1,0,0
4,0,0,1


In [14]:
## One hot encoder columns with the original data

data = pd.concat([data.drop(columns=['Geography'], axis=1), geo_encoded_df], axis=1)

In [15]:
data.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
8057,816,0,34,2,108410.87,2,1,0,102908.91,0,0,1,0
2539,584,0,46,6,87361.02,2,1,0,120376.87,1,0,1,0
4043,635,1,31,1,0.00,2,1,0,135382.23,0,0,0,1
5460,661,1,41,5,122552.48,2,0,1,120646.40,0,0,1,0
8124,791,0,33,10,130229.71,2,0,0,54019.93,1,0,1,0


In [16]:
## Save the encoders and scaler
with open('labe_encoder_gender.pkl', 'wb') as file:
    pickle.dump(labe_encoder_gender, file)

with open('ohe_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)



In [17]:
## divid the dataset into independent and dependent feature

X = data.drop('Exited', axis=1)
y = data['Exited']

## split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [18]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [22]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )), ## first HLL
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

d:\anaconda3\envs\ml_env_latest\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000020AC05345E0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [28]:
## compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [48]:
## Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard 

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [32]:
tensorflow_callback

In [35]:
### Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [41]:
## Train the model


history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8644 - loss: 0.3283 - val_accuracy: 0.8610 - val_loss: 0.3374
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8669 - loss: 0.3251 - val_accuracy: 0.8590 - val_loss: 0.3477
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.8660 - loss: 0.3261 - val_accuracy: 0.8605 - val_loss: 0.3434
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.8680 - loss: 0.3230 - val_accuracy: 0.8540 - val_loss: 0.3416
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8673 - loss: 0.3203 - val_accuracy: 0.8540 - val_loss: 0.3387
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.8677 - loss: 0.3171 - val_accuracy: 0.8570 - val_loss: 0.3539
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8658 - loss: 0.3189 - val_accuracy: 0.8555 - val_loss: 0.3495
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8709 - loss: 0.3164 - 

In [42]:
model.save('model.h5')

In [43]:
## load tensorboard extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [49]:
%tensorboard --logdir logs/fit

In [ ]:
## load the pickle file

